In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import lightning.pytorch as pl
pl.seed_everything(0)

# Set up
*   Install libraries
*   Load dataset

In [ ]:
# ! rm -rf dataset
# ! rm -rf library

In [ ]:
# ! pip install git+https://github.com/IBM/terratorch.git@fix/506

In [ ]:
# ! pip install xarray-spatial

In [ ]:
# ! mkdir -p dataset

In [ ]:
# ! wget https://huggingface.co/datasets/hk-kaden-kim/Small_S2_CloudCover_Seg/resolve/main/train.zip -P dataset
# ! unzip -q dataset/train.zip -d dataset
# ! mv dataset/train dataset/public
# ! rm dataset/train.zip

In [ ]:
# ! wget https://huggingface.co/datasets/hk-kaden-kim/Small_S2_CloudCover_Seg/resolve/main/test.zip -P dataset
# ! unzip -q dataset/test.zip -d dataset
# ! mv dataset/test dataset/private
# ! rm dataset/test.zip

In [ ]:
# ! mkdir -p library
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/__init__.py -P library
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/analysis.py -P library

In [ ]:
# ! mkdir -p library/datasets
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/datasets/__init__.py -P library/datasets
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/datasets/sen2cloud.py -P library/datasets

In [ ]:
# ! mkdir -p library/datamodules
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/datamodules/__init__.py -P library/datamodules
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/datamodules/sen2cloud.py -P library/datamodules

In [ ]:
# ! mkdir -p library/decoders
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/decoders/__init__.py -P library/decoders
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/decoders/unetplusplus.py -P library/decoders

In [ ]:
# ! rm -rf dataset/__MACOSX

# Lightning Trainers

In [ ]:
from library.datamodules.sen2cloud import Sen2CloudDataModule

BATCH_SIZE = 16 # 32

datamodule = Sen2CloudDataModule(
    data_root = './dataset',
    batch_size = BATCH_SIZE,
    means = [2672.63818359375, 2678.138671875, 2587.265380859375, 3507.404052734375],
    stds = [3047.300537109375, 2805.623779296875, 2705.935791015625, 2409.601318359375],
)

datamodule.setup("fit")
datamodule.setup("test")

train_dataset = datamodule.train_dataset
val_dataset = datamodule.val_dataset

test_dataset = datamodule.test_dataset
len(train_dataset), len(val_dataset), len(test_dataset)

In [ ]:
from terratorch.tasks import SemanticSegmentationTask
from library.decoders.unetplusplus import UNetPlusPlusDecoder

LOSS = 'ce'
LEARNING_RATE = 1e-3
OPTIMIZER = 'AdamW'
OPTIMIZER_HPARAMS = {"weight_decay": 0.05}

model_args={
    # Backbone (Encoder)
    "backbone": "prithvi_eo_v1_100", # Model can be either prithvi_eo_v1_100, prithvi_eo_v2_300, prithvi_eo_v2_300_tl, prithvi_eo_v2_600, prithvi_eo_v2_600_tl
    "backbone_kwargs": {
        'pretrained': True,
        'bands': ["BLUE", "GREEN", "RED", "NIR_NARROW"],
        'img_size':512,
    },

    # Necks
    "necks": [
        {
            "name": "SelectIndices",
            "indices": [2, 5, 8, 11] # indices for prithvi_eo_v1_100
            # "indices": [5, 11, 17, 23] # indices for prithvi_eo_v2_300
            # "indices": [7, 15, 23, 31] # indices for prithvi_eo_v2_600
        },
        {"name": "ReshapeTokensToImage",},
        {"name": "LearnedInterpolateToPyramidal"}
    ],

    # Decoder
    "decoder": "UNetDecoder",     # UNetDecoder UNetPlusPlusDecoder
    "decoder_kwargs":{
        'channels': [512, 256, 128, 64],
    },

    # Head
    "head_dropout": 0.1,
    "num_classes": 2,
}

# Model
task = SemanticSegmentationTask(
    model_args=model_args,
    model_factory="EncoderDecoderFactory",
    loss=LOSS,
    lr=LEARNING_RATE,
    optimizer=OPTIMIZER,
    optimizer_hparams=OPTIMIZER_HPARAMS,
    freeze_backbone=True, # True. Only to speed up fine-tuning
    freeze_decoder=False,
    class_names=['No', 'Cloud'],  # optionally define class names
    plot_on_val=0,
)

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint, RichProgressBar
from lightning.pytorch.loggers import TensorBoardLogger

EPOCH = 30

checkpoint_callback = ModelCheckpoint(mode="max",
                                      monitor="val/Multiclass_Jaccard_Index",
                                      filename="best-{epoch:02d}",)
early_stopping_callback = EarlyStopping(mode="min",
                                        monitor=f"val/loss",
                                        # min_delta=0.00001,
                                        patience=10)
logger = TensorBoardLogger(save_dir='output',
                           version=f"E{EPOCH}_B{BATCH_SIZE}_{LOSS}_LR{LEARNING_RATE}",
                           name=f"{model_args['backbone']}_{model_args['decoder']}")

trainer = Trainer(
    devices=1, # Number of GPUs. Interactive mode recommended with 1 device
    precision="16-mixed",
    callbacks=[
      RichProgressBar(),
      checkpoint_callback, # Set to save max val/IoU and last model
      # early_stopping_callback,
      LearningRateMonitor(logging_interval="epoch"),
      ],
    logger=logger,
    max_epochs=EPOCH,
    default_root_dir='output',
    log_every_n_steps=1,
    check_val_every_n_epoch=1,
)

In [ ]:
import torch

torch.cuda.empty_cache()

try:
    free_mem, total_mem = torch.cuda.mem_get_info()
    print(f"Free Memory: {free_mem/1024**2} MB")
    print(f"Total Memory: {total_mem/1024**2} MB")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0)/1024**2} MB")
    print(f"Memory Reserved: {torch.cuda.memory_reserved(0)/1024**2} MB")
    print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated(0)/1024**2} MB")
    print(f"Max Memory Reserved: {torch.cuda.max_memory_reserved(0)/1024**2} MB")
except Exception as e:
    print(e)


In [ ]:
_ = trainer.fit(model=task, datamodule=datamodule)

In [ ]:
res = trainer.test(model=task, datamodule=datamodule) # Check default saved model here

In [ ]:
! rm -rf ./output/prithvi_eo_v1_100_UNetPlusPlusDecoder

In [ ]:
! zip -r prithvi_eo_v1_100_UNetPlusPlusDecoder.zip ./output/prithvi_eo_v1_100_UNetPlusPlusDecoder

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
"""
Backbone : prithvi_eo_v1_100 (Finetune)
Decoder : UNetPlusPlusDecoder (Finetune)
Epoch : 30
Batch : 16
loss : ce
lr : 1e-3
Trainable params: 110 M
Non-trainable params: 0
Total params: 110 M
Total estimated model params size (MB): 443
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test/Multiclass_Accuracy       │         0.8700712323188782          │
│      test/Multiclass_F1_Score       │         0.8700712323188782          │
│    test/Multiclass_Jaccard_Index    │         0.7617184519767761          │
│ test/Multiclass_Jaccard_Index_Micro │         0.7700230479240417          │
│              test/loss              │          0.337007075548172          │
│    test/multiclassaccuracy_Cloud    │         0.8934705853462219          │
│     test/multiclassaccuracy_No      │         0.8342388272285461          │
│  test/multiclassjaccardindex_Cloud  │         0.8062025308609009          │
│   test/multiclassjaccardindex_No    │         0.7172343730926514          │
└─────────────────────────────────────┴─────────────────────────────────────┘

Backbone : prithvi_eo_v1_100
Decoder : UNetPlusPlusDecoder (Finetune)
Epoch : 30
Batch : 16
loss : ce
lr : 1e-3
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test/Multiclass_Accuracy       │         0.8385944366455078          │
│      test/Multiclass_F1_Score       │         0.8385944366455078          │
│    test/Multiclass_Jaccard_Index    │         0.7111126184463501          │
│ test/Multiclass_Jaccard_Index_Micro │         0.7220512628555298          │
│              test/loss              │          0.470979779958725          │
│    test/multiclassaccuracy_Cloud    │         0.8798970580101013          │
│     test/multiclassaccuracy_No      │         0.7753462195396423          │
│  test/multiclassjaccardindex_Cloud  │         0.7673269510269165          │
│   test/multiclassjaccardindex_No    │         0.6548982262611389          │
└─────────────────────────────────────┴─────────────────────────────────────┘

"""

# CLI tool

You find an example for SMP models in `configs/burnscars_smp.yaml` that you can run with `terratorch fit -c configs/burnscars_smp.yaml`.